In [150]:
import pandas as pd
import numpy as np
import os
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, Activation , Flatten,Dropout
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Model
from keras import optimizers
import random
from shutil import copyfile
from keras.models import Sequential
from keras import layers

In [151]:
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256,256,3), pooling=None, classes=1000)

In [152]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [171]:
from keras.optimizers import RMSprop
k = base_model.get_layer(name = 'activation_147')
# Flatten the output layer to 1 dimension
x = layers.Flatten()(k.output)

# Add a dropout rate of 0.2      
x= layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
#x = layers.Dense(3, activation='softmax')(x) 
x = layers.Dense(4, activation='softmax')(x) 

In [172]:
model = Model( base_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [173]:
for layer in model.layers[:-1]:
    if(layer.name.startswith('bn')):
        layer.trainable = True
        layer.momentum = 1
    else:
        layer.trainable = False
for layer in model.layers[-1:]:
    layer.trainable = True   

In [174]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [175]:
for layer in model.layers:
    print(layer.trainable)
    

False
False
False
True
False
False
False
False
True
False
False
True
False
False
False
True
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
False
True
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
False
True
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
False
True
True
False
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
Fal

In [195]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.995):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [196]:
#train_path = '/home/dj/Desktop/Datasets/fcf/train'
#test_path = '/home/dj/Desktop/Datasets/fcf/val'
train_path = '/home/dj/Desktop/Datasets/at_200x/train'
test_path = '/home/dj/Desktop/Datasets/at_200x/val'

In [197]:
train_batches = ImageDataGenerator(rescale=1./255,
    featurewise_center=True,
    zca_whitening=True,
    featurewise_std_normalization=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip = True).flow_from_directory(train_path,target_size=(256,256),shuffle=True,class_mode = 'categorical',batch_size = 20)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,target_size=(256,256),shuffle=True,class_mode = 'categorical',batch_size = 20)

Found 948 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [198]:
callbacks = myCallback()
model.fit_generator(train_batches,steps_per_epoch=10,validation_data = test_batches,epochs=40,validation_steps=50,verbose=1,callbacks=[callbacks])

Epoch 1/40
10/10 [==============================] - 15s 2s/step - loss: 0.2607 - acc: 0.9300 - val_loss: 0.2014 - val_acc: 0.9880
Epoch 2/40
10/10 [==============================] - 13s 1s/step - loss: 0.2924 - acc: 0.9500 - val_loss: 0.2144 - val_acc: 0.9870
Epoch 3/40
10/10 [==============================] - 13s 1s/step - loss: 0.1436 - acc: 0.9450 - val_loss: 0.2394 - val_acc: 0.9740
Epoch 4/40
10/10 [==============================] - 13s 1s/step - loss: 0.2704 - acc: 0.9400 - val_loss: 0.2027 - val_acc: 0.9760
Epoch 5/40
10/10 [==============================] - 12s 1s/step - loss: 0.4156 - acc: 0.9077 - val_loss: 0.2152 - val_acc: 0.9870
Epoch 6/40
10/10 [==============================] - 12s 1s/step - loss: 0.1222 - acc: 0.9800 - val_loss: 0.1946 - val_acc: 0.9880
Epoch 7/40
10/10 [==============================] - 13s 1s/step - loss: 0.3253 - acc: 0.9200 - val_loss: 0.1953 - val_acc: 0.9880
Epoch 8/40
10/10 [==============================] - 13s 1s/step - loss: 0.2177 - acc: 0.92

In [199]:
model.save('Resnet_50_0.975_0.987.h5')

In [200]:
import numpy as np
from keras.preprocessing import image

 
# predicting images
path = '/home/dj/Desktop/testing_folder/turning_200x2.jpg'
img = image.load_img(path, target_size=(256,256,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])/255
classes = model.predict(images, batch_size=10)
print(classes)
categ = ['Buffing','Grinding','Milling','Turning']
print(categ[np.argmax(classes)])

[[  6.50293855e-21   2.15691725e-16   1.60320303e-12   1.00000000e+00]]
Turning


In [201]:
from keras.models import load_model
new = load_model('Resnet_50_0.975_0.987.h5')

In [202]:
import numpy as np
from keras.preprocessing import image

 
# predicting images
path = '/home/dj/Desktop/testing_folder/turning_200x2.jpg'
img = image.load_img(path, target_size=(256,256,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])/255
classes = new.predict(images, batch_size=10)
print(classes)
categ = ['Buffing','Grinding','Milling','Turning']
print(categ[np.argmax(classes)])

[[  6.50293855e-21   2.15691725e-16   1.60320303e-12   1.00000000e+00]]
Turning
